# PART 1 - DataBase Preparation

This will download the necessary files and convert them into two local datasets

In [1]:
import os
from glob import glob
print("importing nilearn masking")
from nilearn import masking
print("successfully imported nilearn masking")
from nimare import dataset, meta
from nimare.extract import fetch_neurosynth
from nimare.io import convert_neurosynth_to_dataset
from nimare.stats import pearson
import pandas as pd
import numpy as np
import os.path as osp
import shutil
import argparse
from tqdm import tqdm

importing nilearn masking


/data/SFIMJGC_HCP7T/Apps/envs/fc_introspection/lib/python3.7/site-packages/nilearn/__init__.py:69: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()


successfully imported nilearn masking


/data/SFIMJGC_HCP7T/Apps/envs/fc_introspection/lib/python3.7/site-packages/nimare/__init__.py:74: FutureWarning: Python 3.6 and 3.7 support is deprecated and will be removed in release 0.1.0 of NiMARE. Consider switching to Python 3.8, 3.9 or 3.10.
  _python_deprecation_warnings()


In [3]:
PRJDIR = "/data/SFIMJGC_Introspec/prj2021_dyneusr/Resources_NiMare/"
vocab = 'LDA50'

In [3]:
RESOURCE_DIR  = osp.join(PRJDIR,vocab)
NIMARE_PATH   = osp.join(RESOURCE_DIR,'nimare')
METAMAPS_DIR  = os.path.join(RESOURCE_DIR,"meta-analyses-orig")  # where to save meta-analysis maps

## 1.1. Create output folders
> **NOTE:** will delete if they already exists

In [ ]:
# Create Empty Output Folders
# ===========================
print("++ INFO: Setting up all necessary folders")
for folder_path in [RESOURCE_DIR, NIMARE_PATH, METAMAPS_DIR]:
    if osp.exists(folder_path):
        print(" + WARNING: Removing folder [%s]" % folder_path)
        shutil.rmtree(folder_path)
    print(" + INFO: Generating/Regenerating output folder [%s]" % folder_path)
    os.mkdir(folder_path)

## 1.2. Download Neurosynth database for the selected vocabulary

This will take approximately 5 mins, but does not have big memory requirements

In [4]:
print("++ INFO: Fetching neurosynth dataset for this vocabulary...")
dset_file    = os.path.join(RESOURCE_DIR, "neurosynth_dataset.pkl.gz")
dset_ma_file = os.path.join(RESOURCE_DIR, "neurosynth_dataset_with_ma.pkl.gz")

++ INFO: Fetching neurosynth dataset for this vocabulary...


In [5]:
files = fetch_neurosynth(
        data_dir=RESOURCE_DIR,
        version="7",
        overwrite=False,
        vocab=vocab)
#,
#        source="abstract",

INFO:nimare.extract.utils:Dataset found in /data/SFIMJGC_Introspec/prj2021_dyneusr/Resources_NiMare/LDA400/neurosynth

INFO:nimare.extract.extract:Searching for any feature files matching the following criteria: [('vocab-LDA400', 'data-neurosynth', 'version-7')]


File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.


In [6]:
neurosynth_db = files[0]

In [7]:
%%time
neurosynth_dset = convert_neurosynth_to_dataset(
        coordinates_file=neurosynth_db["coordinates"],
        metadata_file=neurosynth_db["metadata"],
        annotations_files=neurosynth_db["features"],
        )

CPU times: user 3min, sys: 2.65 s, total: 3min 3s
Wall time: 2min 57s


In [8]:
# Save the dataset as a pickle file to the Resources directory
print (" + Saving dataset to %s" % dset_file)
neurosynth_dset.save(dset_file)

 + Saving dataset to /data/SFIMJGC_Introspec/prj2021_dyneusr/Resources_NiMare/LDA400/neurosynth_dataset.pkl.gz


In [9]:
print(" + Neurosynth dataset: %s" % neurosynth_dset)

 + Neurosynth dataset: Dataset(14371 experiments, space='mni152_2mm')


In [11]:
neurosynth_dset.get_texts()

[]

In [5]:
%%time
if not os.path.isfile(dset_file):
    files = fetch_neurosynth(
        data_dir=RESOURCE_DIR,
        version="7",
        overwrite=False,
        source="abstract",
        vocab=vocab)
    print('=====================')
    print(files)
    print('=====================')
    neurosynth_db = files[0]

    neurosynth_dset = convert_neurosynth_to_dataset(
        coordinates_file=neurosynth_db["coordinates"],
        metadata_file=neurosynth_db["metadata"],
        annotations_files=neurosynth_db["features"],
        )
    # Save the dataset as a pickle file to the Resources directory
    print (" + Saving dataset to %s" % dset_file)
    neurosynth_dset.save(dset_file)
else:
    print (" + Loading dataset from %s" % dset_file)
    neurosynth_dset = dataset.Dataset.load(dset_file)
    
print(" + Neurosynth dataset: %s" % neurosynth_dset)

INFO:nimare.extract.utils:Dataset found in /data/SFIMJGC_Introspec/prj2021_dyneusr/Resources_NiMare/LDA400.interactive/neurosynth

INFO:nimare.extract.extract:Searching for any feature files matching the following criteria: [('source-abstract', 'vocab-LDA400', 'data-neurosynth', 'version-7')]


File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
[{'coordinates': '/data/SFIMJGC_Introspec/prj2021_dyneusr/Resources_NiMare/LDA400.interactive/neurosynth/data-neurosynth_version-7_coordinates.tsv.gz', 'metadata': '/data/SFIMJGC_Introspec/prj2021_dyneusr/Resources_NiMare/LDA400.interactive/neurosynth/data-neurosynth_version-7_metadata.tsv.gz', 'features': [{'features': '/data/SFIMJGC_Introspec/prj2021_dyneusr/Resources_NiMare/LDA400.interactive/neurosynth/data-neurosynth_version-7_vocab-LDA400_source-abstract_type-weight_features.npz', 'vocabulary': '/data/SFIMJGC_Introspec/prj2021_dyneusr/Resources_NiMare/LDA400.interactive/neurosynth/data-neurosynth_version-7_vocab-LDA400_vocabulary.txt', 'keys': '/data/SFIMJGC_Introspec/prj2021_dyneusr/Resources_NiMare/LDA400.i

KeyboardInterrupt: 

In [6]:
files[0]

{'coordinates': '/data/SFIMJGC_Introspec/prj2021_dyneusr/Resources_NiMare/LDA400.interactive/neurosynth/data-neurosynth_version-7_coordinates.tsv.gz',
 'metadata': '/data/SFIMJGC_Introspec/prj2021_dyneusr/Resources_NiMare/LDA400.interactive/neurosynth/data-neurosynth_version-7_metadata.tsv.gz',
 'features': [{'features': '/data/SFIMJGC_Introspec/prj2021_dyneusr/Resources_NiMare/LDA400.interactive/neurosynth/data-neurosynth_version-7_vocab-LDA400_source-abstract_type-weight_features.npz',
   'vocabulary': '/data/SFIMJGC_Introspec/prj2021_dyneusr/Resources_NiMare/LDA400.interactive/neurosynth/data-neurosynth_version-7_vocab-LDA400_vocabulary.txt',
   'keys': '/data/SFIMJGC_Introspec/prj2021_dyneusr/Resources_NiMare/LDA400.interactive/neurosynth/data-neurosynth_version-7_vocab-LDA400_keys.tsv',
   'metadata': '/data/SFIMJGC_Introspec/prj2021_dyneusr/Resources_NiMare/LDA400.interactive/neurosynth/data-neurosynth_version-7_vocab-LDA400_metadata.json'}]}

In [ ]:
# Update Neurosynth dataset path (for meta-analysis intermediate outputs)
print("++ INFO: Updating Neurosynth Object path to [%s]" % NIMARE_PATH)
neurosynth_dset.update_path(NIMARE_PATH)

## 1.3. Convert Neurosynth Database to MA format

In [ ]:
# Initialize the Estimator
print("++ INFO: Initializing estimator")
# You could use `memory_limit` here if you want, but that will slow things down.
# Niamre version 0.0.9 will take the memory_limit input, version 0.0.13 will not
meta_estimator = meta.cbma.mkda.MKDAChi2(memory_limit=ml)

> **NOTE:** This step will require about 100 GB of RAM

In [ ]:
# Pre-generate MA maps to speed the meta-analyses up. This step may take some time.
# This step will create tons of temporary files in NIMARE_PATH
# Independently of memory_limit, it looks like this steps makes mem usage to go up to approx 100GB
# This software runs with NiMare 0.0.9... it breaks with 0.0.13 (current as of March 2023)
print("++ INFO: Pre-generate MA maps to speed the meta-analyses up")
kernel_transformer = meta_estimator.kernel_transformer
neurosynth_dset = kernel_transformer.transform(neurosynth_dset, return_type="dataset")

In [ ]:
# Save neurosynth object updated with MA estimations
print("++ INFO: Saving updated NeuroSynth object [%s]" % dset_ma_file)
neurosynth_dset.save(dset_ma_file)

***
# 2. Generate Meta-Analytic Maps

In [1]:
import os
from glob import glob
print("importing nilearn masking")
from nilearn import masking
print("successfully imported nilearn masking")
from nimare import dataset, meta
from nimare.extract import fetch_neurosynth
from nimare.io import convert_neurosynth_to_dataset
from nimare.stats import pearson
import pandas as pd
import numpy as np
import os.path as osp
import shutil
import argparse
from tqdm import tqdm

importing nilearn masking


/data/SFIMJGC_HCP7T/Apps/envs/fc_introspection/lib/python3.7/site-packages/nilearn/__init__.py:69: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()


successfully imported nilearn masking


/data/SFIMJGC_HCP7T/Apps/envs/fc_introspection/lib/python3.7/site-packages/nimare/__init__.py:74: FutureWarning: Python 3.6 and 3.7 support is deprecated and will be removed in release 0.1.0 of NiMARE. Consider switching to Python 3.8, 3.9 or 3.10.
  _python_deprecation_warnings()


In [2]:
PRJDIR = "/data/SFIMJGC_Introspec/prj2021_dyneusr/Resources_NiMare/"
vocab = 'LDA400'

In [3]:
RESOURCE_DIR  = osp.join(PRJDIR,vocab+".interactive")
NIMARE_PATH   = osp.join(RESOURCE_DIR,'nimare')
METAMAPS_DIR  = os.path.join(RESOURCE_DIR,"meta-analyses-orig")  # where to save meta-analysis maps

In [4]:
print("++ INFO: Fetching neurosynth dataset for this vocabulary...")
dset_file    = os.path.join(RESOURCE_DIR, "neurosynth_dataset.pkl.gz")

++ INFO: Fetching neurosynth dataset for this vocabulary...


In [5]:
neurosynth_dset = dataset.Dataset.load(dset_file)

In [6]:
topics = neurosynth_dset.get_labels()

In [11]:
import pandas as pd
import os.path as osp
import os
from datetime import datetime
import getpass
import subprocess

from utils.basics import get_sbj_scan_list

from utils.basics import PRJ_DIR, DATA_DIR, SCRIPTS_DIR #NOTEBOOKS_DIR, RESOURCES_DINFO_DIR, PREPROCESSING_NOTES_DIR, 
print('++ INFO: Project Dir:                  %s' % PRJ_DIR) 
#print('++ INFO: Notebooks Dir:                %s' % NOTEBOOKS_DIR) 
print('++ INFO: Bash Scripts Dir:             %s' % SCRIPTS_DIR)
#print('++ INFO: Resources (Dataset Info) Dir: %s' % RESOURCES_DINFO_DIR)
#print('++ INFO: Pre-processing Notes Dir:     %s' % PREPROCESSING_NOTES_DIR)
print('++ INFO: Data Dir:                     %s' % DATA_DIR)

++ INFO: Project Dir:                  /data/SFIMJGC_Introspec/2023_fc_introspection
++ INFO: Bash Scripts Dir:             /data/SFIMJGC_Introspec/2023_fc_introspection/code/fc_introspection/bash
++ INFO: Data Dir:                     /data/SFIMJGC_Introspec/pdn


In [12]:
username = getpass.getuser()
print('++ INFO: user working now --> %s' % username)

++ INFO: user working now --> javiergc


In [14]:
#user specific folders
#=====================
swarm_folder   = osp.join(PRJ_DIR,'SwarmFiles.{username}'.format(username=username))
logs_folder    = osp.join(PRJ_DIR,'Logs.{username}'.format(username=username))

swarm_path     = osp.join(swarm_folder,'S19_NiMareTopics.SWARM.sh')
logdir_path    = osp.join(logs_folder, 'S19_NiMareTopics.logs')

In [15]:
# create user specific folders if needed
# ======================================
if not osp.exists(swarm_folder):
    os.makedirs(swarm_folder)
    print('++ INFO: New folder for swarm files created [%s]' % swarm_folder)
if not osp.exists(logdir_path):
    os.makedirs(logdir_path)
    print('++ INFO: New folder for log files created [%s]' % logdir_path)

++ INFO: New folder for log files created [/data/SFIMJGC_Introspec/2023_fc_introspection/Logs.javiergc/S19_NiMareTopics.logs]


In [23]:
# Open the file
swarm_file = open(swarm_path, "w")
# Log the date and time when the SWARM file is created
swarm_file.write('#Create Time: %s' % datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
swarm_file.write('\n')
# Insert comment line with SWARM command
swarm_file.write('#swarm -f {swarm_path} -g 32 -t 8 --partition quick,norm -b 5 --time 00:48:00 --logdir {logdir_path}'.format(swarm_path=swarm_path,logdir_path=logdir_path))
swarm_file.write('\n')

# Insert one line per subject
for topic in topics:
    swarm_file.write(f"export VOCAB={vocab} TOPIC={topic}; sh {SCRIPTS_DIR}/S19_NiMare_Create_TopicMaps.sh")
    swarm_file.write('\n')
swarm_file.close()

In [20]:
print(swarm_path)

/data/SFIMJGC_Introspec/2023_fc_introspection/SwarmFiles.javiergc/S19_NiMareTopics.SWARM.sh


***

In [17]:
from nimare import annotate
from nimare import dataset

In [23]:
dset_file    = os.path.join(RESOURCE_DIR, "neurosynth_dataset.pkl.gz")
dset_ma_file = os.path.join(RESOURCE_DIR, "neurosynth_dataset_with_ma.pkl.gz")

In [24]:
dset_ma = dataset.Dataset.load(dset_ma_file)
dset = dataset.Dataset.load(dset_file)

In [25]:
model = annotate.lda.LDAModel(n_topics=50,max_iter=1000)

In [26]:
dset['vocab']

TypeError: 'Dataset' object is not subscriptable

In [22]:
%%time
new_dset = model.fit(dset_ma)

ValueError: Column 'abstract' not found in DataFrame

In [11]:
neurosynth_dset.get_params()

{'mask': None, 'source': None, 'target': None}

In [ ]:
%%time
# Get features
print ("++ INFO: Get Dictionary topics")
topics = neurosynth_dset.get_labels()
useful_topics = []
for topic in tqdm(topics, desc='Topics'):
    # Use a threshold of 0.05 for topics, even though the default threshold is 0.001.
    topic_positive_ids = neurosynth_dset.get_studies_by_label(topic, 0.05)
    topic_negative_ids = list(set(neurosynth_dset.ids) - set(topic_positive_ids))

    # Require some minimum number of studies in each sample
    if (len(topic_positive_ids) == 0) or (len(topic_negative_ids) == 0):
        print("++ [WARNING] Skipping {topic}".format(topic=topic))
        continue
    useful_topics.append(topic)

    topic_positive_dset = neurosynth_dset.slice(topic_positive_ids)
    topic_negative_dset = neurosynth_dset.slice(topic_negative_ids)
    
    # MEMORY HEAVE LINE
    meta_result = meta_estimator.fit(topic_positive_dset, topic_negative_dset)
    # Save the maps to an output directory
    #print(" +         Saving maps for topic [%s]" % str(topic))
    meta_result.save_maps(output_dir=METAMAPS_DIR, prefix=topic)

++ INFO: Get Dictionary topics


Topics:   0%|▋                                                                                                                                                                                                                                                                    | 1/400 [02:03<13:37:59, 123.01s/it]

In [32]:
meta_result.save_maps(output_dir=METAMAPS_DIR, prefix=topic)

In [33]:
topics

['LDA400_abstract_weight__0_exercise_igd_internet',
 'LDA400_abstract_weight__1_context_contextual_contexts',
 'LDA400_abstract_weight__2_sulcus_intraparietal_ips',
 'LDA400_abstract_weight__3_individuals_resonance_magnetic',
 'LDA400_abstract_weight__4_mood_affective_induction',
 'LDA400_abstract_weight__5_judgments_judgment_judged',
 'LDA400_abstract_weight__6_long_term_short',
 'LDA400_abstract_weight__7_passive_viewing_listening',
 'LDA400_abstract_weight__8_arousal_subjective_ratings',
 'LDA400_abstract_weight__9_amplitude_spontaneous_frequency',
 'LDA400_abstract_weight__10_timing_rhythm_beat',
 'LDA400_abstract_weight__11_children_adults_age',
 'LDA400_abstract_weight__12_structures_pattern_contrast',
 'LDA400_abstract_weight__13_motion_mt_visual',
 'LDA400_abstract_weight__14_central_capacity_nervous',
 'LDA400_abstract_weight__15_pattern_patterns_multivariate',
 'LDA400_abstract_weight__16_executive_control_cognitive',
 'LDA400_abstract_weight__17_pain_painful_chronic',
 'LDA4